<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/llm/kg_construction_llama3groq" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Original notebook: https://github.com/projectwilsen/KnowledgeGraphLLM/blob/main/tutorial/2/notebook.ipynb

YouTube video: https://www.youtube.com/watch?v=ky8LQE-82xs

In [1]:
!pip install --quiet langchain langchain-community langchain-groq neo4j wikipedia tiktoken json-repair

In [2]:
import pandas as pd
import json
import os
import json_repair
from langchain_community.graphs import Neo4jGraph
from langchain_community.chat_models import ChatOllama
from langchain.document_loaders import WikipediaLoader
from langchain_community.llms import Ollama
from langchain.chains import LLMChain
from langchain.prompts.chat import (ChatPromptTemplate,HumanMessagePromptTemplate,SystemMessagePromptTemplate)
from langchain import PromptTemplate
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.schema import (SystemMessage,HumanMessage,AIMessage)
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_groq import ChatGroq
from langchain_community.graphs.graph_document import Node, Relationship, GraphDocument

In [3]:
os.environ["GROQ_API_KEY"] = ""#os.getenv("GROQ_API_KEY")

# Neo4j
neo4j_url = "bolt://44.222.244.170:7687"#os.getenv("NEO4J_CONNECTION_URL")
neo4j_user = "neo4j" #os.getenv("NEO4J_USER")
neo4j_password = "pats" #os.getenv("NEO4J_PASSWORD")

# https://api.python.langchain.com/en/latest/graphs/langchain_community.graphs.neo4j_graph.Neo4jGraph.html
graph = Neo4jGraph(neo4j_url,neo4j_user,neo4j_password)


# Load & Summarize Data

In [4]:
query = "Tim Cook"
raw_documents = WikipediaLoader(query=query).load()
raw_documents

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


[Document(page_content='Timothy Donald Cook (born November 1, 1960) is an American business executive who is the current chief executive officer of Apple Inc. Cook had previously been the company\'s chief operating officer under its co-founder Steve Jobs. Cook joined Apple in March 1998 as a senior vice president for worldwide operations, and then as executive vice president for worldwide sales and operations. He was appointed chief executive on August 24, 2011 after Jobs, who was ill and died that October, resigned. During his tenure as the chief executive, he has advocated for the political reform of international and domestic surveillance, cybersecurity, American manufacturing, and environmental preservation. \nSince 2011 when he took over Apple, to 2020, Cook doubled the company\'s revenue and profit, and the company\'s market value increased from $348 billion to $1.9 trillion. Cook is also on the boards of directors of Nike, Inc. and the National Football Foundation; he is a trust

In [5]:
filtered_raw_documents = [raw_documents[i] for i in [0,1,4,7,8,9,10,12,13]] #0: Tim Cook (person), 1: Apple (company), 4: Mac (product), 10: Research, 11: Apple Maps, 13: App Store, 7: Apple TV, 8: Steve Jobs, 13: iPhone
docs = " ".join([d.page_content for d in filtered_raw_documents]).replace("\n", "").replace("==", "")
print(docs)

Timothy Donald Cook (born November 1, 1960) is an American business executive who is the current chief executive officer of Apple Inc. Cook had previously been the company's chief operating officer under its co-founder Steve Jobs. Cook joined Apple in March 1998 as a senior vice president for worldwide operations, and then as executive vice president for worldwide sales and operations. He was appointed chief executive on August 24, 2011 after Jobs, who was ill and died that October, resigned. During his tenure as the chief executive, he has advocated for the political reform of international and domestic surveillance, cybersecurity, American manufacturing, and environmental preservation. Since 2011 when he took over Apple, to 2020, Cook doubled the company's revenue and profit, and the company's market value increased from $348 billion to $1.9 trillion. Cook is also on the boards of directors of Nike, Inc. and the National Football Foundation; he is a trustee of Duke University, his al

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=30
)
split_docs = text_splitter.create_documents([docs])

# Extract Information

In [7]:
entity_types = ['person','school','award','company','product','characteristic']
relation_types = ['alumniOf','worksFor','hasAward','isProducedBy','hasCharacteristic','acquired','hasProject','isFounderOf']

system_prompt = PromptTemplate(
    template = """
    You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph.
    Your task is to identify the entities and relations requested with the user prompt, from a given text.
    You must generate the output in a JSON containing a list with JSON objects having the following keys: "head", "head_type", "relation", "tail", and "tail_type".
    The "head" key must contain the text of the extracted entity with one of the types from the provided list in the user prompt.
    The "head_type" key must contain the type of the extracted head entity which must be one of the types from {entity_types}.
    The "relation" key must contain the type of relation between the "head" and the "tail" which must be one of the relations from {relation_types}.
    The "tail" key must represent the text of an extracted entity which is the tail of the relation, and the "tail_type" key must contain the type of the tail entity from {entity_types}.
    Attempt to extract as many entities and relations as you can.

    IMPORTANT NOTES:
    - Don't add any explanation and text.
    """,
    input_variables=["entity_types","relation_types"],
)


system_message_prompt = SystemMessagePromptTemplate(prompt = system_prompt)

examples = [
        {
            "text":"Adam is a software engineer in Microsoft since 2009, and last year he got an award as the Best Talent" ,
            "head": "Adam",
            "head_type": "person",
            "relation": "worksFor",
            "tail": "Microsoft",
            "tail_type": "company"
        },
        {
            "text":"Adam is a software engineer in Microsoft since 2009, and last year he got an award as the Best Talent" ,
            "head": "Adam",
            "head_type": "person",
            "relation": "hasAward",
            "tail": "Best Talent",
            "tail_type": "award"
        },
        {
            "text":"Microsoft is a tech company that provide several products such as Microsoft Word" ,
            "head": "Microsoft Word",
            "head_type": "product",
            "relation": "isproducedBy",
            "tail": "Microsoft",
            "tail_type": "company"
        },
        {
            "text":"Microsoft Word is a lightweight app that accessible offline" ,
            "head": "Microsoft Word",
            "head_type": "product",
            "relation": "hasCharacteristic",
            "tail": "lightweight app",
            "tail_type": "characteristic"
        },
        {
            "text":"Microsoft Word is a lightweight app that accessible offline" ,
            "head": "Microsoft Word",
            "head_type": "product",
            "relation": "hasCharacteristic",
            "tail": "accesible offline",
            "tail_type": "characteristic"
        },
    ]

class ExtractedInfo(BaseModel):
    head: str = Field(description="extracted first or head entity like Microsoft, Apple, John")
    head_type: str = Field(description="type of the extracted head entity like person, company, etc")
    relation: str = Field(description="relation between the head and the tail entities")
    tail: str = Field(description="extracted second or tail entity like Microsoft, Apple, John")
    tail_type: str = Field(description="type of the extracted tail entity like person, company, etc")

parser = JsonOutputParser(pydantic_object=ExtractedInfo)

human_prompt = PromptTemplate(
    template = """ Based on the following example, extract entities and relations from the provided text.\n\n

    Use the following entity types, don't use other entity that is not defined below:
    # ENTITY TYPES:
    {entity_types}

    Use the following relation types, don't use other relation that is not defined below:
    # RELATION TYPES:
    {relation_types}

    Below are a number of examples of text and their extracted entities and relationshhips.
    {examples}

    For the following text, generate extract entitites and relations as in the provided example.\n{format_instructions}\nText: {text}""",
    input_variables=["entity_types","relation_types","examples","text"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

human_message_prompt = HumanMessagePromptTemplate(prompt=human_prompt)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
model = ChatGroq(temperature=0, model_name="llama3-70b-8192")
chain = chat_prompt | model

In [8]:
graph_docs = []
for doc in split_docs[:3]: # only 3 docs
    response  = chain.invoke({"entity_types" : entity_types, "relation_types" : relation_types, "examples" : examples, "text" : doc.page_content})
    relationships = []
    for rel in json_repair.loads(response.content):
        source_node = Node(id=rel["head"], type=rel["head_type"])
        target_node = Node(id=rel["tail"], type=rel["tail_type"])
        relationships.append(Relationship(source=source_node, target=target_node, type=rel['relation']))

    graph_document = GraphDocument(
        nodes=[],
        relationships=relationships,
        source=doc
    )
    graph_docs.append(graph_document)

In [9]:
graph.add_graph_documents(graph_docs)